In [87]:
import cv2
import os
from keras import layers, Input
from keras.models import Model
from keras import models
from keras import optimizers
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import os
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
import numpy as np

In [8]:
total_data_final_2 = pd.read_csv('./total_data_final_2.csv')
total_data_final_2.columns
df = total_data_final_2[['folder_name','image_num','category_label','category_name',
                         'category_type','texture','shape','style']]


In [56]:
# category_type = 3 삭제
df = df[df.category_type != 3]
df.shape

(198558, 8)

In [57]:
df

,folder_name,image_num,category_label,category_name,category_type,texture,shape,style
0,Sheer_Pleated-Front_Blouse,img_00000001.jpg,3,Blouse,1,NaN,shift,NaN
1,Sheer_Pleated-Front_Blouse,img_00000002.jpg,3,Blouse,1,NaN,shift,NaN
2,Sheer_Pleated-Front_Blouse,img_00000003.jpg,3,Blouse,1,NaN,slip,NaN
3,Sheer_Pleated-Front_Blouse,img_00000004.jpg,3,Blouse,1,NaN,NaN,NaN
4,Sheer_Pleated-Front_Blouse,img_00000005.jpg,3,Blouse,1,NaN,NaN,shark
...,...,...,...,...,...,...,...,...
198553,Destroyed_Denim_Cutoffs,img_00000015.jpg,24,Cutoffs,2,diamond,distressed skinny,NaN
198554,Destroyed_Denim_Cutoffs,img_00000016.jpg,24,Cutoffs,2,diamond print,denim pencil,elegant
198555,Destroyed_Denim_Cutoffs,img_00000017.jpg,24,Cutoffs,2,diamond print,denim pencil,rolling
198556,Destroyed_Denim_Cutoffs,img_00000018.jpg,24,Cutoffs,2,paint,distressed low-rise,NaN


In [81]:
# 폴더 생성 함수

def createfolder(foler_dir):
    try:
        if not os.path.exists(folder_dir):
            os.makedirs(folder_dir)
    except OSError:
        print('Error Creating directory'+folder_dir)


In [82]:
# category_name 폴더 만들기

category_list = list(df['category_name'].unique())

for folder_name in category_list:
    a = 'C:/Users/student/Desktop/category/'
    folder_dir = a + folder_name
    createfolder(folder_dir)

In [83]:
# category_name에 맞춰서 사진 다시 저장

for _ in range(df.shape[0]):
    num = str(_)
    category_name = df['category_name'][_]
    img_folder = df['folder_name'][_] 
    img_name = df['image_num'][_] 
    full_dir = str('D:/Project/img/' + img_folder + '/' + img_name) 
    
    img = cv2.imread(full_dir)
    img = cv2.resize(img, (150,150))
    
    new_dir = str('C:/Users/student/Desktop/category/'+ category_name)
    new_name = new_dir + '/' + category_name + '_' + num + '.jpg'
    cv2.imwrite(new_name, img) 
    
    if _ % 5000 == 0:
        print(_)


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000


## category_name으로 학습

In [84]:
generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    rescale = 1. / 255)

In [90]:
model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same', use_bias=False, input_shape=(150,150, 3)))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(32, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))
 
model.add(Conv2D(128, (3, 3), padding='same', use_bias=False))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dropout(0.2))
 
model.add(Dense(512, activation='relu'))
model.add(Dense(36, activation='softmax'))
model.summary()
 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 150, 150, 16)      432       
_________________________________________________________________
batch_normalization_5 (Batch (None, 150, 150, 16)      48        
_________________________________________________________________
activation_5 (Activation)    (None, 150, 150, 16)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 38, 38, 16)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 38, 38, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 38, 38, 32)        4608      
_________________________________________________________________
batch_normalization_6 (Batch (None, 38, 38, 32)        96        
__________

In [110]:
# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


# 데이터 셋
train_datagen = ImageDataGenerator(rescale=1./255,
#                                   rotation_range=10,
#                                   width_shift_range=0.2,
#                                   height_shift_range=0.2,
#                                   shear_range=0.7,
#                                   zoom_range=[0.9, 2.2],
#                                   horizontal_flip=True,
#                                   vertical_flip=True,
#                                   fill_mode='nearest',
                                   validation_split=0.20)
 
training_set = train_datagen.flow_from_directory('C:/Users/student/Desktop/category',
                                                 target_size = (150,150),
                                                 batch_size = 15,
                                                 class_mode = 'categorical',
                                                 subset="training"
                                                )
validation_set = train_datagen.flow_from_directory('C:/Users/student/Desktop/category',
                                                 target_size = (150,150),
                                                 batch_size = 10,
                                                 class_mode = 'categorical',
                                                 subset="validation"
                                                  )
 

Found 158860 images belonging to 36 classes.
Found 39698 images belonging to 36 classes.


In [ ]:
from keras.callbacks import CSVLogger
 
csv_logger = CSVLogger('./log.csv', append=True, separator=';')
 
 
hist = model.fit_generator(training_set,
                         steps_per_epoch = 20, #20
                         epochs = 15000,  #1000
                         validation_data = validation_set,
                         validation_steps = 10,#10
                         callbacks=[csv_logger])
 
 

Epoch 1/15000
20/20 [==============================] - 15s 729ms/step - loss: 1.9796 - acc: 0.3700 - val_loss: 2.0967 - val_acc: 0.2900
Epoch 2/15000
13/20 [==================>...........] - ETA: 4s - loss: 1.8935 - acc: 0.4000

In [ ]:
from keras.models import load_model
 
model.save('cnn_attraction_keras_model.h5')
 
# output = classifier.predict_generator(test_set, steps=5)
# print(test_set.class_indices)
# print(output)
 
# 모델 평가하기
print("-- Evaluate --")
 
scores = model.evaluate_generator(
            validation_set,
            steps = 10)
 
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
 
# 모델 예측하기
print("-- Predict --")
 
output = model.predict_generator(
            validation_set,
            steps = 100)
print(validation_set.class_indices)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
 

In [ ]:
import matplotlib.pyplot as plt
 
fig, loss_ax = plt.subplots()
 
acc_ax = loss_ax.twinx()
 
loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
#loss_ax.set_ylim([0.0, 0.5])
 
acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
#acc_ax.set_ylim([0.8, 1.0])
 
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')
 
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
 
plt.show()